In [1]:
from sacremoses import MosesDetokenizer
import pandas as pd
import numpy as np
import csv

In [2]:
data = pd.read_csv('data/questions.tsv', sep='\t', engine='python', warn_bad_lines=True, quoting=csv.QUOTE_NONE)

/var/folders/kt/nshm9j555pn7z_t0hxzy3v4h0000gn/T/ipykernel_15712/4041008024.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('data/questions.tsv', sep='\t', engine='python', warn_bad_lines=True, quoting=csv.QUOTE_NONE)


In [3]:
md = MosesDetokenizer(lang='en')

In [4]:
def moses_detokenize(sentence):
    return md.detokenize(sentence)

def dumb_quotes(x):
    return (x
        .replace('\u201c', '"').replace('\u201d', '"') # double quotes
        .replace('\u2018', "'").replace('\u2019', "'") # single quotes
    )

def dumb_spaces(sentence):
    return (sentence
            .replace(" ' ", "'")
            .replace(" - ", "-"))

In [5]:
def preprocess_sent(sentence):
    return moses_detokenize(dumb_spaces(dumb_quotes(sentence)).split())

In [6]:
data['Sentence'] = data['Sentence'].apply(preprocess_sent)

In [26]:
def get_cumulative_context(sentences): 
    current_ctx = ""
    context = []

    for sent in sentences.unique():
        current_ctx += (sent + " ")
        context.append(current_ctx)
    
    return context

In [27]:
def do_thing_to_group(article_data): 
    return article_data.assign(Context=get_cumulative_context(article_data['Sentence']))

In [30]:
new_df = data.groupby(['Article_Id', "Sentence_Id"]).head(1).groupby("Article_Id").apply(do_thing_to_group)

In [39]:
new_df['Context'].iloc[0]

"The nation's largest gun-rights group is taking some Texans to task over their headline-generating demonstrations advocating the legal, open carrying of weapons. "

In [42]:
new_df_context_only = new_df.filter(["Article_Id", "Sentence_Id", "Context"], axis=1)
new_df_context_only

,Article_Id,Sentence_Id,Context
0,1,1,The nation's largest gun-rights group is takin...
4,1,2,The nation's largest gun-rights group is takin...
6,1,3,The nation's largest gun-rights group is takin...
9,1,4,The nation's largest gun-rights group is takin...
10,1,5,The nation's largest gun-rights group is takin...
...,...,...,...
19804,1500,1,"It's the underdog of U. S. currency, the green..."
19807,1500,2,"It's the underdog of U. S. currency, the green..."
19809,1500,3,"It's the underdog of U. S. currency, the green..."
19811,1500,4,"It's the underdog of U. S. currency, the green..."


In [44]:
joined_context_new = pd.merge(data, new_df_context_only, on=["Article_Id", "Sentence_Id"])
joined_context_new

,Article_Id,Sentence_Id,Sentence,Span,Question,Span_Start_Position,Span_End_Position,Context_x,Context_y
0,1,1,The nation's largest gun-rights group is takin...,to task,"What does \""to task\"" mean?",13,15,The nation's largest gun-rights group is takin...,The nation's largest gun-rights group is takin...
1,1,1,The nation's largest gun-rights group is takin...,largest gun - rights group,What is this group called?,4,9,The nation's largest gun-rights group is takin...,The nation's largest gun-rights group is takin...
2,1,1,The nation's largest gun-rights group is takin...,gun - rights group,Which group?,5,9,The nation's largest gun-rights group is takin...,The nation's largest gun-rights group is takin...
3,1,1,The nation's largest gun-rights group is takin...,nation ’ s largest gun - rights group,Why don't you just come out and say the NRA?,1,9,The nation's largest gun-rights group is takin...,The nation's largest gun-rights group is takin...
4,1,2,Officials with the National Rifle Association ...,small number,How many people is a small number?,67,69,The nation's largest gun-rights group is takin...,The nation's largest gun-rights group is takin...
...,...,...,...,...,...,...,...,...,...
19811,1500,4,John Bennardo is crisscrossing the country to ...,"the story of the two and its "" magic """,Why are $2 bills seen as so much more desirable?,14,24,"It's the underdog of U. S. currency, the green...","It's the underdog of U. S. currency, the green..."
19812,1500,4,John Bennardo is crisscrossing the country to ...,""" magic """,What magic are they referring to?,21,24,"It's the underdog of U. S. currency, the green...","It's the underdog of U. S. currency, the green..."
19813,1500,5,"""I think everyone's curious about it,"" he said.","everyone ' s curious about it , """,Why is everyone so curious about it?,3,11,"It's the underdog of U. S. currency, the green...","It's the underdog of U. S. currency, the green..."
19814,1500,5,"""I think everyone's curious about it,"" he said.",everyone ' s curious,Why do they feel everyone would be curious abo...,3,7,"It's the underdog of U. S. currency, the green...","It's the underdog of U. S. currency, the green..."


In [29]:
data['Context'] = data.groupby('Article_Id')['Sentence'].apply(lambda x: (x + ' ').cumsum().str.strip())


0        The nation's largest gun-rights group is takin...
1        The nation's largest gun-rights group is takin...
2        The nation's largest gun-rights group is takin...
3        The nation's largest gun-rights group is takin...
4        The nation's largest gun-rights group is takin...
                               ...                        
19811    It's the underdog of U. S. currency, the green...
19812    It's the underdog of U. S. currency, the green...
19813    It's the underdog of U. S. currency, the green...
19814    It's the underdog of U. S. currency, the green...
19815    It's the underdog of U. S. currency, the green...
Name: Context, Length: 19816, dtype: object

In [ ]:
article = data[data['Article_Id'] == 1]
print(article)

sentences = article['Sentence'].unique()

current_ctx = ""
context = []

for sent in sentences: 
    current_ctx += sent
    context.append(current_ctx)

context

In [ ]:
article_groups = data.groupby(["Article_Id", "Sentence_Id"])
sample = article_groups.get_group((1, 1))
sample.loc["Context"] = 1

In [ ]:
for article_id, article_data in data.groupby("Article_Id"):
    


In [ ]:
article = data[data['Article_Id'] == 1]
print(article)

sentences = article['Sentence'].unique()

current_ctx = ""
context = []

for sent in sentences: 
    current_ctx += sent
    context.append(current_ctx)

conditions = [
    article['Sentence_Id'] == 1,
    article['Sentence_Id'] == 2,
    article['Sentence_Id'] == 3,
    article['Sentence_Id'] == 4,
    article['Sentence_Id'] == 5
]

article['Context'] = np.select(conditions, context)


In [ ]:
article['Context'] = np.select(conditions, context)
article.Context[6]
assert len(data.Sentence_Id.unique()) == 5

In [ ]:
groupby_articles = data.groupby("Article_Id")


In [ ]:
groupby_articles.cumsum()